In [1]:
import argparse
import Classifier
import joblib
import Util
import json
import Model
import os
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from tqdm import tqdm
from Util import *
import os

In [34]:
args = {
    'list' : 'log-titles.csv',
    'model' : '../models-article-classifier/model-05-02-2020-0.99831.joblib'
}

N_JOBS = 6                                                                          


# #LENDOS OS TEXTO
print('Readind texts of:',args['list'])
texts = []
labels = []

data = []
with open(args['list'],'r') as f:
    for line in f:
        
        line = line.replace('\n','')
        line = line.split(';')
        aux = {
            'file': line[0],            
            'label': line[1],
            'title':line[2],
            'url':line[3]
        }
        if os.path.isfile(line[0]):
            with open(line[0]) as _f:
                line[0] = _f.read()
                aux['content'] = line[0]
        else:
            aux['content'] = line[0]
        if 'content' in aux:
            data.append(aux)
        
        

#CARREGANDO O MODELO
print('Loading model!')
model = Model.Model()
model.loadModel(args['model'])
print('Model Accuraci:',model.acc)


MAX = len(data)
aux = Parallel(n_jobs=N_JOBS)(delayed(analiser)(data[i]['content']) for i in tqdm(range(MAX)))
t = []
for i in range(len(data)):    
    if aux[i]:
        data[i]['lang'] = [aux[i][0]]
    else:
        t.append(i)

t.reverse()
print('Removendo os items que a lingua não possuimos modelos!')
for i in t:
    del data[i]

MAX = len(data)
t = []
aux = Parallel(n_jobs=N_JOBS)(delayed(filtering)(data[i]['content'],data[i]['lang']) for i in tqdm(range(MAX)))
for i in range(len(data)):  
    data[i]['text'] = aux[i]
    if not aux[i]:
        t.append(i)

t.reverse()
print('Removendo os items que não geram textos tokenizados!')
for i in t:
    del data[i]

MAX = len(data)

aux = Parallel(n_jobs=N_JOBS)(delayed(TextCounter)(data[i]['text']) for i in tqdm(range(MAX)))
for i in range(len(data)):    
    data[i]['words'] = aux[i]

texts = [i['words'] for i in data]
labels = []
for i in data:
    labels.append(model.label_map[i['label']])

vets = Util.Vectorize(model.dictionary,texts)


  0%|          | 0/598 [00:00<?, ?it/s]

Readind texts of: log-titles.csv
Loading model!
Model Accuraci: 0.9983050847457627



100%|██████████| 598/598 [00:33<00:00, 17.89it/s]

 14%|█▍        | 84/591 [00:00<00:00, 821.68it/s]

Removendo os items que a lingua não possuimos modelos!



100%|██████████| 591/591 [00:00<00:00, 1441.89it/s]

100%|██████████| 590/590 [00:00<00:00, 16702.59it/s]

Removendo os items que não geram textos tokenizados!


In [38]:
y_pred = []
for i in range(len(vets)):
    vet = vets[i]
    _id = model.classifier.algorithm.predict([vet])[0]    
    prob = model.classifier.algorithm.predict_proba([vet])[0]
    y_pred.append(_id)
    for label, val in model.label_map.items():
        diff = abs(prob[0] - prob[1])
        if val == _id and label == 'não':
            if diff < 0.30 or diff > 0.60:
#                 print(label,data[i]['label'],diff)
                print(data[i]['title'],';',data[i]['url'])
#             print('--------------')
# print('Acc: ',accuracy_score(labels,y_pred))

PRODUÇÃO DE ROCHA ARTIFICIAL UTILIZANDO RESÍDUO DE ETAPA DE LAMINAÇÃO DE VIDRO PLANO EM MATRIZ DE RESINA POLIMÉRICA ; http://uenf.br/posgraduacao/engenharia-de-materiais/wp-content/uploads/sites/2/2013/07/DISSERTA%C3%87%C3%83O-COMPLETA-Lucas-Martins-final.pdf
Caracterização física e mecânica de rocha artificial com resíduo de pó de vidro sódico-cálcico e resina poliéster insaturada por meio de moldagem por pressagem ; http://www2.ufcg.edu.br/revista-remap/index.php/REMAP/article/viewArticle/691
Análise da reutilização do vidro moído para fabricação de mármore sintético ; http://repositorio.roca.utfpr.edu.br/jspui/handle/1/12631
CARACTERIZAÇÃO E MOAGEM DE VIDRO PROVENIENTE DO RESÍDUO DA TRITURAÇÃO DE SUCATA EM SHREDDER DE INDÚSTRIA METALÚRGICA ; http://cbecimat.com.br/anais/PDF/Ig07-001.pdf
ANÁLISE AMBIENTAL E DE MAQUINÁRIO DE MARMORARIAS NO MUNICÍPIO DE PALMAS-TO ; http://ferramentas.unipinhal.edu.br/engenhariaambiental/include/getdoc.php?id=2985&article=877&mode=pdf
Geração de sílica 